In [134]:
import requests as r
from bs4 import BeautifulSoup as bs
import urllib
import json
import time
import unicodedata

def dic_p12(key, var):
    ''' saca value de la key recurrentemente comm p12'''
    if hasattr(var,'items'):
        for k, v in var.items():
            if k == key:
                yield unicodedata.normalize('NFKC',bs(v).text).replace('\n',' ')
            if isinstance(v, dict):
                for result in dic_p12(key, v):
                    yield unicodedata.normalize('NFKC',bs(result).text).replace('\n',' ')
            elif isinstance(v, list):
                for d in v:
                    for result in dic_p12(key, d):
                        yield unicodedata.normalize('NFKC',bs(result).text).replace('\n',' ')
                        
                        
class p12():
    
    def __init__(self):
        self.url='https://www.pagina12.com.ar'
    #poner a mano el limite de comentarios, por default en la pagina=250 en qp12

    
    def get(self,url):
        nota=r.get(url)
        sopa=bs(nota.content,features="lxml")
        ps=sopa.find('div','article-inner padding-right').findAll('p')
        st=sopa.find('div','article-inner padding-right').findAll('b')
        try:
            self.volanta=sopa.find('h2','article-prefix').text
        except:
            pass
        self.titulo=sopa.find('h1').text
        try:
            self.bajada=sopa.find('div','article-summary').text
        except:
            pass
        texto=list()
        for p in ps:
            texto.append(p.text)
        bolds=list()    
        for b in st:
            bolds.append(b.text)            
        self.bold=' '.join(bolds)
        self.bolds=bolds    
        self.cuerpo=' '.join(texto)
        with open('qp12', 'r') as file:
            qp12 = file.read()
        payload=json.loads(qp12)
        aid=url.split('/')[-1].split('-')[0]
        payload['variables']['assetId']=aid
        payload['variables']['assetUrl']=url
        payload.pop('operationName')
        pp=r.post("https://talk.pagina12.com.ar/api/v1/graph/ql", json=payload)
        coms=[]
        for x in pp.json()['data']['asset']['comments']['nodes']:
            coms.extend(list(dic_p12('body',x)))        

        self.coms=coms

        
    def hoy(self):            
        notas=r.get(self.url)
        sopa=bs(notas.content,features="lxml")
        urls=[x.find('a').get('href') for x in sopa.find_all('article')]
        self.urls=urls

        
def get(urls):
    notas=list()
    for url in urls:
        if url[:28]=='https://www.pagina12.com.ar/':
            nota=p12()
            try:
                nota.get(url)
                notas.append(nota)
                time.sleep(0.4)
            except:
                print(url)
        else:
            pass
    return(notas)


def getv(url):
    nota=r.get(url)
    sopa=bs(nota.content,features="lxml")
    ps=sopa.find('div','article-inner padding-right').findAll('p')
    st=sopa.find('div','article-inner padding-right').findAll('b')
    volanta=sopa.find('h2','article-prefix').text
    titulo=sopa.find('h1').text
    try:
        bajada=sopa.find('div','article-summary').text
    except:
        pass
    texto=list()
    for p in ps:
        texto.append(p.text)
    bolds=list()    
    for b in st:
        bolds.append(b.text)            
    bold=' '.join(bolds)
    cuerpo=' '.join(texto)
    aid=url.split('/')[-1].split('-')[0]
    x=r'{"query":"query CoralEmbedStream_Embed($assetId: ID, $assetUrl: String, $commentId: ID!, $hasComment: Boolean!, $excludeIgnored: Boolean, $sortBy: SORT_COMMENTS_BY!, $sortOrder: SORT_ORDER!) {\n  me {\n    id\n    state {\n      status {\n        username {\n          status\n          __typename\n        }\n        banned {\n          status\n          __typename\n        }\n        alwaysPremod {\n          status\n          __typename\n        }\n        suspension {\n          until\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  asset(id: $assetId, url: $assetUrl) {\n    ...CoralEmbedStream_Configure_asset\n    ...CoralEmbedStream_Stream_asset\n    ...CoralEmbedStream_AutomaticAssetClosure_asset\n    __typename\n  }\n  ...CoralEmbedStream_Stream_root\n  ...CoralEmbedStream_Configure_root\n}\n\nfragment CoralEmbedStream_Stream_root on RootQuery {\n  me {\n    state {\n      status {\n        username {\n          status\n          __typename\n        }\n        banned {\n          status\n          __typename\n        }\n        alwaysPremod {\n          status\n          __typename\n        }\n        suspension {\n          until\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    ignoredUsers {\n      id\n      __typename\n    }\n    role\n    __typename\n  }\n  settings {\n    organizationName\n    __typename\n  }\n  ...TalkSlot_StreamTabPanes_root\n  ...TalkSlot_StreamFilter_root\n  ...CoralEmbedStream_Comment_root\n  __typename\n}\n\nfragment CoralEmbedStream_Comment_root on RootQuery {\n  me {\n    ignoredUsers {\n      id\n      __typename\n    }\n    __typename\n  }\n  ...TalkSlot_CommentInfoBar_root\n  ...TalkSlot_CommentAuthorName_root\n  ...TalkEmbedStream_DraftArea_root\n  ...TalkEmbedStream_DraftArea_root\n  __typename\n}\n\nfragment TalkEmbedStream_DraftArea_root on RootQuery {\n  __typename\n}\n\nfragment CoralEmbedStream_Stream_asset on Asset {\n  comment(id: $commentId) @include(if: $hasComment) {\n    ...CoralEmbedStream_Stream_comment\n    parent {\n      ...CoralEmbedStream_Stream_singleComment\n      parent {\n        ...CoralEmbedStream_Stream_singleComment\n        parent {\n          ...CoralEmbedStream_Stream_singleComment\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  id\n  title\n  url\n  isClosed\n  created_at\n  settings {\n    moderation\n    infoBoxEnable\n    infoBoxContent\n    premodLinksEnable\n    questionBoxEnable\n    questionBoxContent\n    questionBoxIcon\n    closedTimeout\n    closedMessage\n    disableCommenting\n    disableCommentingMessage\n    charCountEnable\n    charCount\n    requireEmailConfirmation\n    __typename\n  }\n  totalCommentCount @skip(if: $hasComment)\n  comments(query: {limit: 50, excludeIgnored: $excludeIgnored, sortOrder: $sortOrder, sortBy: $sortBy}) @skip(if: $hasComment) {\n    nodes {\n      ...CoralEmbedStream_Stream_comment\n      __typename\n    }\n    hasNextPage\n    startCursor\n    endCursor\n    __typename\n  }\n  ...TalkSlot_StreamTabsPrepend_asset\n  ...TalkSlot_StreamTabPanes_asset\n  ...TalkSlot_StreamFilter_asset\n  ...CoralEmbedStream_Comment_asset\n  __typename\n}\n\nfragment CoralEmbedStream_Comment_asset on Asset {\n  __typename\n  id\n  ...TalkSlot_CommentInfoBar_asset\n  ...TalkSlot_CommentActions_asset\n  ...TalkSlot_CommentReactions_asset\n  ...TalkSlot_CommentAuthorName_asset\n}\n\nfragment CoralEmbedStream_Stream_comment on Comment {\n  id\n  status\n  user {\n    id\n    __typename\n  }\n  ...CoralEmbedStream_Comment_comment\n  __typename\n}\n\nfragment CoralEmbedStream_Comment_comment on Comment {\n  ...CoralEmbedStream_Comment_SingleComment\n  replies(query: {limit: 3, excludeIgnored: $excludeIgnored}) {\n    nodes {\n      ...CoralEmbedStream_Comment_SingleComment\n      replies(query: {limit: 3, excludeIgnored: $excludeIgnored}) {\n        nodes {\n          ...CoralEmbedStream_Comment_SingleComment\n          replies(query: {limit: 3, excludeIgnored: $excludeIgnored}) {\n            nodes {\n              ...CoralEmbedStream_Comment_SingleComment\n              __typename\n            }\n            hasNextPage\n            startCursor\n            endCursor\n            __typename\n          }\n          __typename\n        }\n        hasNextPage\n        startCursor\n        endCursor\n        __typename\n      }\n      __typename\n    }\n    hasNextPage\n    startCursor\n    endCursor\n    __typename\n  }\n  __typename\n}\n\nfragment CoralEmbedStream_Comment_SingleComment on Comment {\n  id\n  body\n  created_at\n  status\n  replyCount\n  tags {\n    tag {\n      name\n      __typename\n    }\n    __typename\n  }\n  user {\n    id\n    username\n    __typename\n  }\n  status_history {\n    type\n    __typename\n  }\n  action_summaries {\n    __typename\n    count\n    current_user {\n      id\n      __typename\n    }\n  }\n  editing {\n    edited\n    editableUntil\n    __typename\n  }\n  ...TalkSlot_CommentInfoBar_comment\n  ...TalkSlot_CommentActions_comment\n  ...TalkSlot_CommentReactions_comment\n  ...TalkSlot_CommentAuthorName_comment\n  ...TalkSlot_CommentContent_comment\n  ...TalkEmbedStream_DraftArea_comment\n  ...TalkEmbedStream_DraftArea_comment\n  __typename\n}\n\nfragment TalkEmbedStream_DraftArea_comment on Comment {\n  __typename\n  ...TalkSlot_DraftArea_comment\n}\n\nfragment CoralEmbedStream_Stream_singleComment on Comment {\n  id\n  status\n  user {\n    id\n    __typename\n  }\n  ...CoralEmbedStream_Comment_SingleComment\n  __typename\n}\n\nfragment CoralEmbedStream_Configure_root on RootQuery {\n  __typename\n  ...CoralEmbedStream_Settings_root\n}\n\nfragment CoralEmbedStream_Settings_root on RootQuery {\n  __typename\n}\n\nfragment CoralEmbedStream_Configure_asset on Asset {\n  __typename\n  ...CoralEmbedStream_AssetStatusInfo_asset\n  ...CoralEmbedStream_Settings_asset\n}\n\nfragment CoralEmbedStream_AssetStatusInfo_asset on Asset {\n  id\n  closedAt\n  isClosed\n  __typename\n}\n\nfragment CoralEmbedStream_Settings_asset on Asset {\n  id\n  settings {\n    moderation\n    premodLinksEnable\n    questionBoxEnable\n    questionBoxIcon\n    questionBoxContent\n    __typename\n  }\n  __typename\n}\n\nfragment CoralEmbedStream_AutomaticAssetClosure_asset on Asset {\n  id\n  closedAt\n  __typename\n}\n\nfragment TalkSlot_StreamTabPanes_root on RootQuery {\n  ...TalkFeaturedComments_TabPane_root\n  __typename\n}\n\nfragment TalkFeaturedComments_TabPane_root on RootQuery {\n  __typename\n  ...TalkFeaturedComments_Comment_root\n}\n\nfragment TalkFeaturedComments_Comment_root on RootQuery {\n  __typename\n  ...TalkSlot_CommentAuthorName_root\n}\n\nfragment TalkSlot_StreamFilter_root on RootQuery {\n  ...TalkViewingOptions_ViewingOptions_root\n  __typename\n}\n\nfragment TalkViewingOptions_ViewingOptions_root on RootQuery {\n  __typename\n}\n\nfragment TalkSlot_CommentInfoBar_root on RootQuery {\n  ...TalkModerationActions_root\n  __typename\n}\n\nfragment TalkModerationActions_root on RootQuery {\n  me {\n    id\n    __typename\n  }\n  __typename\n}\n\nfragment TalkSlot_CommentAuthorName_root on RootQuery {\n  ...TalkAuthorMenu_AuthorName_root\n  __typename\n}\n\nfragment TalkAuthorMenu_AuthorName_root on RootQuery {\n  __typename\n  ...TalkSlot_AuthorMenuActions_root\n}\n\nfragment TalkSlot_StreamTabsPrepend_asset on Asset {\n  ...TalkFeaturedComments_Tab_asset\n  __typename\n}\n\nfragment TalkFeaturedComments_Tab_asset on Asset {\n  featuredCommentsCount: totalCommentCount(tags: [\"FEATURED\"]) @skip(if: $hasComment)\n  __typename\n}\n\nfragment TalkSlot_StreamTabPanes_asset on Asset {\n  ...TalkFeaturedComments_TabPane_asset\n  __typename\n}\n\nfragment TalkFeaturedComments_TabPane_asset on Asset {\n  id\n  featuredComments: comments(query: {tags: [\"FEATURED\"], sortOrder: $sortOrder, sortBy: $sortBy, excludeIgnored: $excludeIgnored}, deep: true) @skip(if: $hasComment) {\n    nodes {\n      ...TalkFeaturedComments_Comment_comment\n      __typename\n    }\n    hasNextPage\n    startCursor\n    endCursor\n    __typename\n  }\n  ...TalkFeaturedComments_Comment_asset\n  __typename\n}\n\nfragment TalkFeaturedComments_Comment_comment on Comment {\n  id\n  body\n  created_at\n  replyCount\n  tags {\n    tag {\n      name\n      __typename\n    }\n    __typename\n  }\n  user {\n    id\n    username\n    __typename\n  }\n  ...TalkSlot_CommentReactions_comment\n  ...TalkSlot_CommentAuthorName_comment\n  ...TalkSlot_CommentContent_comment\n  __typename\n}\n\nfragment TalkFeaturedComments_Comment_asset on Asset {\n  __typename\n  ...TalkSlot_CommentReactions_asset\n  ...TalkSlot_CommentAuthorName_asset\n}\n\nfragment TalkSlot_StreamFilter_asset on Asset {\n  ...TalkViewingOptions_ViewingOptions_asset\n  __typename\n}\n\nfragment TalkViewingOptions_ViewingOptions_asset on Asset {\n  __typename\n}\n\nfragment TalkSlot_CommentInfoBar_asset on Asset {\n  ...TalkModerationActions_asset\n  __typename\n}\n\nfragment TalkModerationActions_asset on Asset {\n  id\n  __typename\n}\n\nfragment TalkSlot_CommentActions_asset on Asset {\n  ...TalkPermalink_Button_asset\n  __typename\n}\n\nfragment TalkPermalink_Button_asset on Asset {\n  url\n  __typename\n}\n\nfragment TalkSlot_CommentReactions_asset on Asset {\n  ...RespectButton_asset\n  __typename\n}\n\nfragment RespectButton_asset on Asset {\n  id\n  __typename\n}\n\nfragment TalkSlot_CommentAuthorName_asset on Asset {\n  ...TalkAuthorMenu_AuthorName_asset\n  __typename\n}\n\nfragment TalkAuthorMenu_AuthorName_asset on Asset {\n  __typename\n}\n\nfragment TalkSlot_CommentInfoBar_comment on Comment {\n  ...TalkFeaturedComments_Tag_comment\n  ...TalkModerationActions_comment\n  __typename\n}\n\nfragment TalkFeaturedComments_Tag_comment on Comment {\n  tags {\n    tag {\n      name\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment TalkModerationActions_comment on Comment {\n  id\n  status\n  user {\n    id\n    __typename\n  }\n  tags {\n    tag {\n      name\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment TalkSlot_CommentActions_comment on Comment {\n  ...TalkPermalink_Button_comment\n  __typename\n}\n\nfragment TalkPermalink_Button_comment on Comment {\n  id\n  __typename\n}\n\nfragment TalkSlot_CommentReactions_comment on Comment {\n  ...RespectButton_comment\n  __typename\n}\n\nfragment RespectButton_comment on Comment {\n  id\n  action_summaries {\n    __typename\n    ... on RespectActionSummary {\n      count\n      current_user {\n        id\n        __typename\n      }\n      __typename\n    }\n  }\n  __typename\n}\n\nfragment TalkSlot_CommentAuthorName_comment on Comment {\n  ...TalkAuthorMenu_AuthorName_comment\n  __typename\n}\n\nfragment TalkAuthorMenu_AuthorName_comment on Comment {\n  __typename\n  id\n  user {\n    username\n    __typename\n  }\n  ...TalkSlot_AuthorMenuInfos_comment\n  ...TalkSlot_AuthorMenuActions_comment\n}\n\nfragment TalkSlot_CommentContent_comment on Comment {\n  ...TalkPluginRichText_CommentContent_comment\n  __typename\n}\n\nfragment TalkPluginRichText_CommentContent_comment on Comment {\n  body\n  richTextBody\n  __typename\n}\n\nfragment TalkSlot_DraftArea_comment on Comment {\n  ...TalkPluginRichText_Editor_comment\n  __typename\n}\n\nfragment TalkPluginRichText_Editor_comment on Comment {\n  body\n  richTextBody\n  __typename\n}\n\nfragment TalkSlot_AuthorMenuActions_root on RootQuery {\n  ...TalkIgnoreUser_IgnoreUserAction_root\n  __typename\n}\n\nfragment TalkIgnoreUser_IgnoreUserAction_root on RootQuery {\n  me {\n    id\n    __typename\n  }\n  __typename\n}\n\nfragment TalkSlot_AuthorMenuInfos_comment on Comment {\n  ...TalkMemberSince_MemberSinceInfo_comment\n  __typename\n}\n\nfragment TalkMemberSince_MemberSinceInfo_comment on Comment {\n  user {\n    username\n    created_at\n    __typename\n  }\n  __typename\n}\n\nfragment TalkSlot_AuthorMenuActions_comment on Comment {\n  ...TalkIgnoreUser_IgnoreUserAction_comment\n  __typename\n}\n\nfragment TalkIgnoreUser_IgnoreUserAction_comment on Comment {\n  user {\n    id\n    __typename\n  }\n  ...TalkIgnoreUser_IgnoreUserConfirmation_comment\n  __typename\n}\n\nfragment TalkIgnoreUser_IgnoreUserConfirmation_comment on Comment {\n  user {\n    id\n    username\n    __typename\n  }\n  __typename\n}\n","variables":{"assetId":"300200","assetUrl":"https://www.pagina12.com.ar/300200-elecciones-bolivia-2020-carlos-mesa-reconocio-la-derrota","commentId":"","hasComment":false,"excludeIgnored":false,"sortBy":"CREATED_AT","sortOrder":"DESC"},"operationName":"CoralEmbedStream_Embed"}'
    payload=json.loads(x)
    payload['variables']['assetId']=aid
    payload['variables']['assetUrl']=url
    pp=r.post("https://talk.pagina12.com.ar/api/v1/graph/ql", json=payload)
    coms=pp.json()['data']['asset']['comments']['nodes']
    comm=[unicodedata.normalize("NFKD",x['body'].strip().replace('\n','')) for x in coms]
    com=' '.join(comm)
    print(volanta)
    print(titulo)
    
    return(comm)

In [131]:
nota=p12()
nota.get('https://www.pagina12.com.ar/310594-larreta-renuncia-al-pacto-fiscal-que-firmo-con-macri-para-po')

In [132]:
nota.coms

['Soy lego en la materia, que alguien me lo confirme: ¿es cierto que hay empresas que tienen su actividad en provincias o el interior pero tributan por domicilio fiscal en CABA? ¿De qué otra manera se transfieren ingresos, más allá de los impuestos? porque donde hay desigualdad pronunciada, hay  alguna "trampa"- No existe persona ni entidad que, por más laborioso y eficiente que sea, pueda acumular 10, 20, 500 veces más que su vecino en real igualdad de condiciones. ',
 'La CABA tiene el gobierno que votaron desde hace más de una década. Solo lo pueden revertir en las próximas elecciones.',
 'Coincido....  muy buena la foto de El Guason....Larreta...., en CABA....!!!!;🤣🤣🤣🤡🤡',
 'Estoy de acuerdo con Maximiliano Ferraro.....  Llamenlo a Macri y su entorno ministerial y económico si es que quieren recuperar lo robado. Si lo logran, no tienen que ir mas a la Corte a presionar para que les devuelva lo que Macri les regaló. ',
 'Reforma Constitucional y cambio del asiento de la Capital Nacio

In [ ]:
x=r'{"query":"query CoralEmbedStream_Embed($assetId: ID, $assetUrl: String, $commentId: ID!, $hasComment: Boolean!, $excludeIgnored: Boolean, $sortBy: SORT_COMMENTS_BY!, $sortOrder: SORT_ORDER!) {\n  me {\n    id\n    state {\n      status {\n        username {\n          status\n          __typename\n        }\n        banned {\n          status\n          __typename\n        }\n        alwaysPremod {\n          status\n          __typename\n        }\n        suspension {\n          until\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  asset(id: $assetId, url: $assetUrl) {\n    ...CoralEmbedStream_Configure_asset\n    ...CoralEmbedStream_Stream_asset\n    ...CoralEmbedStream_AutomaticAssetClosure_asset\n    __typename\n  }\n  ...CoralEmbedStream_Stream_root\n  ...CoralEmbedStream_Configure_root\n}\n\nfragment CoralEmbedStream_Stream_root on RootQuery {\n  me {\n    state {\n      status {\n        username {\n          status\n          __typename\n        }\n        banned {\n          status\n          __typename\n        }\n        alwaysPremod {\n          status\n          __typename\n        }\n        suspension {\n          until\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    ignoredUsers {\n      id\n      __typename\n    }\n    role\n    __typename\n  }\n  settings {\n    organizationName\n    __typename\n  }\n  ...TalkSlot_StreamTabPanes_root\n  ...TalkSlot_StreamFilter_root\n  ...CoralEmbedStream_Comment_root\n  __typename\n}\n\nfragment CoralEmbedStream_Comment_root on RootQuery {\n  me {\n    ignoredUsers {\n      id\n      __typename\n    }\n    __typename\n  }\n  ...TalkSlot_CommentInfoBar_root\n  ...TalkSlot_CommentAuthorName_root\n  ...TalkEmbedStream_DraftArea_root\n  ...TalkEmbedStream_DraftArea_root\n  __typename\n}\n\nfragment TalkEmbedStream_DraftArea_root on RootQuery {\n  __typename\n}\n\nfragment CoralEmbedStream_Stream_asset on Asset {\n  comment(id: $commentId) @include(if: $hasComment) {\n    ...CoralEmbedStream_Stream_comment\n    parent {\n      ...CoralEmbedStream_Stream_singleComment\n      parent {\n        ...CoralEmbedStream_Stream_singleComment\n        parent {\n          ...CoralEmbedStream_Stream_singleComment\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  id\n  title\n  url\n  isClosed\n  created_at\n  settings {\n    moderation\n    infoBoxEnable\n    infoBoxContent\n    premodLinksEnable\n    questionBoxEnable\n    questionBoxContent\n    questionBoxIcon\n    closedTimeout\n    closedMessage\n    disableCommenting\n    disableCommentingMessage\n    charCountEnable\n    charCount\n    requireEmailConfirmation\n    __typename\n  }\n  totalCommentCount @skip(if: $hasComment)\n  comments(query: {limit: 250, excludeIgnored: $excludeIgnored, sortOrder: $sortOrder, sortBy: $sortBy}) @skip(if: $hasComment) {\n    nodes {\n      ...CoralEmbedStream_Stream_comment\n      __typename\n    }\n    hasNextPage\n    startCursor\n    endCursor\n    __typename\n  }\n  ...TalkSlot_StreamTabsPrepend_asset\n  ...TalkSlot_StreamTabPanes_asset\n  ...TalkSlot_StreamFilter_asset\n  ...CoralEmbedStream_Comment_asset\n  __typename\n}\n\nfragment CoralEmbedStream_Comment_asset on Asset {\n  __typename\n  id\n  ...TalkSlot_CommentInfoBar_asset\n  ...TalkSlot_CommentActions_asset\n  ...TalkSlot_CommentReactions_asset\n  ...TalkSlot_CommentAuthorName_asset\n}\n\nfragment CoralEmbedStream_Stream_comment on Comment {\n  id\n  status\n  user {\n    id\n    __typename\n  }\n  ...CoralEmbedStream_Comment_comment\n  __typename\n}\n\nfragment CoralEmbedStream_Comment_comment on Comment {\n  ...CoralEmbedStream_Comment_SingleComment\n  replies(query: {limit: 25, excludeIgnored: $excludeIgnored}) {\n    nodes {\n      ...CoralEmbedStream_Comment_SingleComment\n      replies(query: {limit: 25, excludeIgnored: $excludeIgnored}) {\n        nodes {\n          ...CoralEmbedStream_Comment_SingleComment\n          replies(query: {limit: 25, excludeIgnored: $excludeIgnored}) {\n            nodes {\n              ...CoralEmbedStream_Comment_SingleComment\n              __typename\n            }\n            hasNextPage\n            startCursor\n            endCursor\n            __typename\n          }\n          __typename\n        }\n        hasNextPage\n        startCursor\n        endCursor\n        __typename\n      }\n      __typename\n    }\n    hasNextPage\n    startCursor\n    endCursor\n    __typename\n  }\n  __typename\n}\n\nfragment CoralEmbedStream_Comment_SingleComment on Comment {\n  id\n  body\n  created_at\n  status\n  replyCount\n  tags {\n    tag {\n      name\n      __typename\n    }\n    __typename\n  }\n  user {\n    id\n    username\n    __typename\n  }\n  status_history {\n    type\n    __typename\n  }\n  action_summaries {\n    __typename\n    count\n    current_user {\n      id\n      __typename\n    }\n  }\n  editing {\n    edited\n    editableUntil\n    __typename\n  }\n  ...TalkSlot_CommentInfoBar_comment\n  ...TalkSlot_CommentActions_comment\n  ...TalkSlot_CommentReactions_comment\n  ...TalkSlot_CommentAuthorName_comment\n  ...TalkSlot_CommentContent_comment\n  ...TalkEmbedStream_DraftArea_comment\n  ...TalkEmbedStream_DraftArea_comment\n  __typename\n}\n\nfragment TalkEmbedStream_DraftArea_comment on Comment {\n  __typename\n  ...TalkSlot_DraftArea_comment\n}\n\nfragment CoralEmbedStream_Stream_singleComment on Comment {\n  id\n  status\n  user {\n    id\n    __typename\n  }\n  ...CoralEmbedStream_Comment_SingleComment\n  __typename\n}\n\nfragment CoralEmbedStream_Configure_root on RootQuery {\n  __typename\n  ...CoralEmbedStream_Settings_root\n}\n\nfragment CoralEmbedStream_Settings_root on RootQuery {\n  __typename\n}\n\nfragment CoralEmbedStream_Configure_asset on Asset {\n  __typename\n  ...CoralEmbedStream_AssetStatusInfo_asset\n  ...CoralEmbedStream_Settings_asset\n}\n\nfragment CoralEmbedStream_AssetStatusInfo_asset on Asset {\n  id\n  closedAt\n  isClosed\n  __typename\n}\n\nfragment CoralEmbedStream_Settings_asset on Asset {\n  id\n  settings {\n    moderation\n    premodLinksEnable\n    questionBoxEnable\n    questionBoxIcon\n    questionBoxContent\n    __typename\n  }\n  __typename\n}\n\nfragment CoralEmbedStream_AutomaticAssetClosure_asset on Asset {\n  id\n  closedAt\n  __typename\n}\n\nfragment TalkSlot_StreamTabPanes_root on RootQuery {\n  ...TalkFeaturedComments_TabPane_root\n  __typename\n}\n\nfragment TalkFeaturedComments_TabPane_root on RootQuery {\n  __typename\n  ...TalkFeaturedComments_Comment_root\n}\n\nfragment TalkFeaturedComments_Comment_root on RootQuery {\n  __typename\n  ...TalkSlot_CommentAuthorName_root\n}\n\nfragment TalkSlot_StreamFilter_root on RootQuery {\n  ...TalkViewingOptions_ViewingOptions_root\n  __typename\n}\n\nfragment TalkViewingOptions_ViewingOptions_root on RootQuery {\n  __typename\n}\n\nfragment TalkSlot_CommentInfoBar_root on RootQuery {\n  ...TalkModerationActions_root\n  __typename\n}\n\nfragment TalkModerationActions_root on RootQuery {\n  me {\n    id\n    __typename\n  }\n  __typename\n}\n\nfragment TalkSlot_CommentAuthorName_root on RootQuery {\n  ...TalkAuthorMenu_AuthorName_root\n  __typename\n}\n\nfragment TalkAuthorMenu_AuthorName_root on RootQuery {\n  __typename\n  ...TalkSlot_AuthorMenuActions_root\n}\n\nfragment TalkSlot_StreamTabsPrepend_asset on Asset {\n  ...TalkFeaturedComments_Tab_asset\n  __typename\n}\n\nfragment TalkFeaturedComments_Tab_asset on Asset {\n  featuredCommentsCount: totalCommentCount(tags: [\"FEATURED\"]) @skip(if: $hasComment)\n  __typename\n}\n\nfragment TalkSlot_StreamTabPanes_asset on Asset {\n  ...TalkFeaturedComments_TabPane_asset\n  __typename\n}\n\nfragment TalkFeaturedComments_TabPane_asset on Asset {\n  id\n  featuredComments: comments(query: {tags: [\"FEATURED\"], sortOrder: $sortOrder, sortBy: $sortBy, excludeIgnored: $excludeIgnored}, deep: true) @skip(if: $hasComment) {\n    nodes {\n      ...TalkFeaturedComments_Comment_comment\n      __typename\n    }\n    hasNextPage\n    startCursor\n    endCursor\n    __typename\n  }\n  ...TalkFeaturedComments_Comment_asset\n  __typename\n}\n\nfragment TalkFeaturedComments_Comment_comment on Comment {\n  id\n  body\n  created_at\n  replyCount\n  tags {\n    tag {\n      name\n      __typename\n    }\n    __typename\n  }\n  user {\n    id\n    username\n    __typename\n  }\n  ...TalkSlot_CommentReactions_comment\n  ...TalkSlot_CommentAuthorName_comment\n  ...TalkSlot_CommentContent_comment\n  __typename\n}\n\nfragment TalkFeaturedComments_Comment_asset on Asset {\n  __typename\n  ...TalkSlot_CommentReactions_asset\n  ...TalkSlot_CommentAuthorName_asset\n}\n\nfragment TalkSlot_StreamFilter_asset on Asset {\n  ...TalkViewingOptions_ViewingOptions_asset\n  __typename\n}\n\nfragment TalkViewingOptions_ViewingOptions_asset on Asset {\n  __typename\n}\n\nfragment TalkSlot_CommentInfoBar_asset on Asset {\n  ...TalkModerationActions_asset\n  __typename\n}\n\nfragment TalkModerationActions_asset on Asset {\n  id\n  __typename\n}\n\nfragment TalkSlot_CommentActions_asset on Asset {\n  ...TalkPermalink_Button_asset\n  __typename\n}\n\nfragment TalkPermalink_Button_asset on Asset {\n  url\n  __typename\n}\n\nfragment TalkSlot_CommentReactions_asset on Asset {\n  ...RespectButton_asset\n  __typename\n}\n\nfragment RespectButton_asset on Asset {\n  id\n  __typename\n}\n\nfragment TalkSlot_CommentAuthorName_asset on Asset {\n  ...TalkAuthorMenu_AuthorName_asset\n  __typename\n}\n\nfragment TalkAuthorMenu_AuthorName_asset on Asset {\n  __typename\n}\n\nfragment TalkSlot_CommentInfoBar_comment on Comment {\n  ...TalkFeaturedComments_Tag_comment\n  ...TalkModerationActions_comment\n  __typename\n}\n\nfragment TalkFeaturedComments_Tag_comment on Comment {\n  tags {\n    tag {\n      name\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment TalkModerationActions_comment on Comment {\n  id\n  status\n  user {\n    id\n    __typename\n  }\n  tags {\n    tag {\n      name\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment TalkSlot_CommentActions_comment on Comment {\n  ...TalkPermalink_Button_comment\n  __typename\n}\n\nfragment TalkPermalink_Button_comment on Comment {\n  id\n  __typename\n}\n\nfragment TalkSlot_CommentReactions_comment on Comment {\n  ...RespectButton_comment\n  __typename\n}\n\nfragment RespectButton_comment on Comment {\n  id\n  action_summaries {\n    __typename\n    ... on RespectActionSummary {\n      count\n      current_user {\n        id\n        __typename\n      }\n      __typename\n    }\n  }\n  __typename\n}\n\nfragment TalkSlot_CommentAuthorName_comment on Comment {\n  ...TalkAuthorMenu_AuthorName_comment\n  __typename\n}\n\nfragment TalkAuthorMenu_AuthorName_comment on Comment {\n  __typename\n  id\n  user {\n    username\n    __typename\n  }\n  ...TalkSlot_AuthorMenuInfos_comment\n  ...TalkSlot_AuthorMenuActions_comment\n}\n\nfragment TalkSlot_CommentContent_comment on Comment {\n  ...TalkPluginRichText_CommentContent_comment\n  __typename\n}\n\nfragment TalkPluginRichText_CommentContent_comment on Comment {\n  body\n  richTextBody\n  __typename\n}\n\nfragment TalkSlot_DraftArea_comment on Comment {\n  ...TalkPluginRichText_Editor_comment\n  __typename\n}\n\nfragment TalkPluginRichText_Editor_comment on Comment {\n  body\n  richTextBody\n  __typename\n}\n\nfragment TalkSlot_AuthorMenuActions_root on RootQuery {\n  ...TalkIgnoreUser_IgnoreUserAction_root\n  __typename\n}\n\nfragment TalkIgnoreUser_IgnoreUserAction_root on RootQuery {\n  me {\n    id\n    __typename\n  }\n  __typename\n}\n\nfragment TalkSlot_AuthorMenuInfos_comment on Comment {\n  ...TalkMemberSince_MemberSinceInfo_comment\n  __typename\n}\n\nfragment TalkMemberSince_MemberSinceInfo_comment on Comment {\n  user {\n    username\n    created_at\n    __typename\n  }\n  __typename\n}\n\nfragment TalkSlot_AuthorMenuActions_comment on Comment {\n  ...TalkIgnoreUser_IgnoreUserAction_comment\n  __typename\n}\n\nfragment TalkIgnoreUser_IgnoreUserAction_comment on Comment {\n  user {\n    id\n    __typename\n  }\n  ...TalkIgnoreUser_IgnoreUserConfirmation_comment\n  __typename\n}\n\nfragment TalkIgnoreUser_IgnoreUserConfirmation_comment on Comment {\n  user {\n    id\n    username\n    __typename\n  }\n  __typename\n}\n","variables":{"assetId":"","assetUrl":"","commentId":"","hasComment":false,"excludeIgnored":false,"sortBy":"CREATED_AT","sortOrder":"DESC"},"operationName":"CoralEmbedStream_Embed"}'


In [127]:
url='https://www.pagina12.com.ar/310594-larreta-renuncia-al-pacto-fiscal-que-firmo-con-macri-para-po'
with open('qp12', 'r') as file:
    qp12 = file.read()
payload=json.loads(qp12)
aid=url.split('/')[-1].split('-')[0]
payload['variables']['assetId']=aid
payload['variables']['assetUrl']=url
payload.pop('operationName')
pp=r.post("https://talk.pagina12.com.ar/api/v1/graph/ql", json=payload)
coms=[]
for x in pp.json()['data']['asset']['comments']['nodes']:
    coms.extend(list(dict_extract('body',x)))
len(coms)

29

In [128]:
coms

['Soy lego en la materia, que alguien me lo confirme: ¿es cierto que hay empresas que tienen su actividad en provincias o el interior pero tributan por domicilio fiscal en CABA? ¿De qué otra manera se transfieren ingresos, más allá de los impuestos? porque donde hay desigualdad pronunciada, hay\xa0 alguna "trampa"- No existe persona ni entidad que, por más laborioso y eficiente que sea, pueda acumular 10, 20, 500 veces más que su vecino en real igualdad de condiciones.\xa0',
 'La CABA tiene el gobierno que votaron desde hace más de una década. Solo lo pueden revertir en las próximas elecciones.',
 'Coincido....\xa0 muy buena la foto de El Guason....Larreta...., en CABA....!!!!;🤣🤣🤣🤡🤡',
 'Estoy de acuerdo con Maximiliano Ferraro.....\xa0 Llamenlo a Macri y su entorno ministerial y económico si es que quieren recuperar lo robado. Si lo logran, no tienen que ir mas a la Corte a presionar para que les devuelva lo que Macri les regaló.\xa0',
 'Reforma Constitucional y cambio del asiento de l

'CoralEmbedStream_Embed'

In [19]:
response.text

'POST body missing. Did you forget use body-parser middleware?'

In [2]:
req=r.get('https://www.pagina12.com.ar/')

In [4]:
sopa=bs(req.content)

In [6]:
arts=sopa.find_all('article')

In [15]:
arts[0].text

'Estafas, vaciamiento, desvío de fondos, ventas simuladas y empresas pantalla bajo investigación judicialLos Etchevehere: un modelo ejemplar de apropiación privadaLas maniobras de ocultamiento de activos en perjuicio de la hermana del ex ministro, pero también de acreedores y otros vínculos, están bajo la lupa de la Justicia. Las operaciones que ya le imputó la Fiscalía al clan familiar.Por Raúl Dellatorre'

In [10]:
arts[-1].text

'Festival Internacional de Cortos de Juyuy Este lunes 26 de octubre comienza la 19° edición de la renombrada muestra, de forma online y mediante la plataforma OctubreTV'

In [12]:
len(arts)

133

In [19]:
arts[0].find('a').get('href')

'https://www.pagina12.com.ar/302979-los-etchevehere-el-modelo-ejemplar-de-apropiacion-privada'

### hoy

In [28]:
pa12=p12()

In [29]:
pa12.hoy()

In [30]:
pa12.urls

['https://www.pagina12.com.ar/302979-los-etchevehere-el-modelo-ejemplar-de-apropiacion-privada',
 'https://www.pagina12.com.ar/302958-coronavirus-en-argentina-se-registraron-211-muertes',
 'https://www.pagina12.com.ar/302948-elecciones-en-estados-unidos-el-analisis-de-noam-chomsky-sob',
 'https://www.pagina12.com.ar/302971-el-proyecto-artigas-sigue-en-marcha',
 'https://www.pagina12.com.ar/302937-europa-en-crisis-por-la-segunda-ola-del-virus',
 'https://www.pagina12.com.ar/302801-el-golpe-que-trump-puede-dar-para-seguir-en-el-poder-aunque-',
 'https://www.pagina12.com.ar/302963-cornejo-cruzo-a-carrio-y-salio-a-defender-a-macri',
 'https://www.pagina12.com.ar/302965-femicidio-en-tucuman-un-hombre-mato-a-una-mujer-que-acosaba-',
 'https://www.pagina12.com.ar/302192-que-es-la-luna-azul-el-extrano-fenomeno-que-podra-verse-este',
 'https://www.pagina12.com.ar/302791-en-netflix-no-se-consigue',
 'https://www.pagina12.com.ar/302840-escandalo-en-la-causa-del-dolar-futuro-uno-de-los-jueces-adv'

In [31]:
len('https://www.pagina12.com.ar/')

28

In [70]:
notas=get(pa12.urls)

https://www.pagina12.com.ar/autores/824-eric-nepomuceno
https://www.pagina12.com.ar/autores/863-juan-jose-panno
https://www.pagina12.com.ar/autores/262061-esther-diaz


In [65]:
len(notas)

118